<a href="https://colab.research.google.com/github/shivammehta007/Information-Retrieval/blob/master/TextRelevancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Relevancy Without Boolean Search

### Some Operations for Google Colab
Mounting Drive and Downloading Data From Kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Download Data
!pip install kaggle
import json
import os

with open('/content/drive/My Drive/kaggle.json') as kc:
    kaggle_config = json.load(kc)

os.environ['KAGGLE_USERNAME'] = kaggle_config['username'] 
os.environ['KAGGLE_KEY'] = kaggle_config['key'] 

!kaggle competitions download -c text-relevance-competition-ir-2-itmo-fall-2019
!unzip docs.tsv.zip
!rm docs.tsv.zip

queries.numerate.txt: Skipping, found more recently modified local copy (use --force to force download)
 99% 1.15G/1.17G [00:09<00:00, 139MB/s]
100% 1.17G/1.17G [00:09<00:00, 132MB/s]
sample.submission.txt: Skipping, found more recently modified local copy (use --force to force download)
Archive:  docs.tsv.zip
  inflating: docs.tsv                


In [4]:
!pip install -U tqdm
!pip install rank_bm25
!pip install keyw
import os
import gzip
import nltk
import random
import pickle
import pandas as pd
import keyw
import logging
import itertools
import re
import time

import numpy as np
import pandas as pd

from rank_bm25 import BM25Okapi

from nltk.corpus import stopwords
nltk.download("stopwords")

from string import punctuation
from collections import defaultdict
from collections import OrderedDict
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


logging.basicConfig(level=logging.DEBUG)
logging.debug('Test Logger')

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.41.1)


DEBUG:root:Test Logger


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Adding Preprocessing of data

In [0]:
stemmer = nltk.SnowballStemmer("russian")
stop_words = set(stopwords.words("russian")) | set(stopwords.words("english")) | set(['это'])

In [0]:
def preprocess(sentence):
    """
    Method to remove stop words and punctuations return tokens
    """
    sentence = keyw.engrus(sentence)
    NONTEXT = re.compile('[^0-9 a-z#+_а-яё]')

    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', punctuation))
    sentence = re.sub(NONTEXT,'',sentence)

    sentence = set(sentence.split())

    tokens = sentence - stop_words
    # tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

In [187]:
# Testing of function:
preprocess('бандиты боятся ли ментов 123 &^% the is как это')

'бандиты ментов 123 боятся еру шы'

# Get Query and Their Relevant Documents

In [0]:
queries_location = '/content/queries.numerate.txt'
database_location = '/content/docs.tsv'
relevant_docs_location = '/content/sample.submission.txt'

In [0]:
queries = pd.read_csv(queries_location, sep='\t', names=['QueryId', 'Query']).applymap(str)
relevant_docs = pd.read_csv(relevant_docs_location).applymap(str)

In [0]:
# Modifying those tables to have only one information
queries['Query'] = queries['Query'].apply(preprocess)
relevant_docs = pd.DataFrame(relevant_docs.groupby('QueryId')['DocumentId'].agg(','.join)).reset_index()
relevance_data = pd.merge(queries, relevant_docs, left_on='QueryId', right_on='QueryId', how='inner')

In [191]:
relevance_data.head()

,QueryId,Query,DocumentId
0,1,отрицательный положительный 1 могут,"28590,28847,33229,33711,36240,36527,38598,4174..."
1,31,ящик удаленный почтовый востановить,"21651,26501,26511,26514,30864,100675,140009,14..."
2,33,квест удалить скайрим,"16658,20060,31777,45320,45638,45664,48088,5250..."
3,44,попасть папку темп,"11664,12590,31019,33810,35647,41030,57731,6182..."
4,46,иглой двойной шить,"3763,24823,65720,66255,68069,68278,80150,80151..."


### Loading the database

In [127]:
%%time
database = pd.read_csv(database_location, sep='\t', header=None,
                       names=['id', 'subject', 'content'],
                       dtype = {'id': int,'subject': str,
                            'content': str})

CPU times: user 57.4 s, sys: 22.9 s, total: 1min 20s
Wall time: 1min 20s


In [173]:
database.shape

(37816, 3)

## Initially Using TFIDF Cosine Similarities

In [0]:
# All relevant documents, will use later after preprocessing them first

# all_relevant_documents = np.array([np.array(x.split(',')) for x in .values])
# len(all_relevant_documents.flatten())



In [194]:
with open('submission.csv', 'w') as output:
    output.write('QueryId,DocumentId\n')
    for index, row in tqdm(relevance_data.iterrows()):
        query = row['Query']
        id = row['QueryId']
        relevant_documents = row['DocumentId'].split(',')
        data = database[database['id'].isin(list(relevant_documents))].copy()
        data.loc[:, 'content'] = data[['subject', 'content']].astype(str).apply(' '.join, axis=1)
        del data['subject']
        data = OrderedDict(data.set_index('id')['content'].to_dict())


        tfidf = TfidfVectorizer()
        ids = data.keys()
        data = tfidf.fit_transform(data.values())
        query = tfidf.transform([query])[0]
        distances = cosine_similarity(query, data)
        rankings =  sorted(list(zip(ids, distances[0])), key=lambda x: x[1], reverse=True)[:10] 
        for rank in rankings:
            output.write('{},{}\n'.format(id, rank[0]))
        
        data = None
        

It Gave me **0.66791** Score on Kaggle

In [205]:
with open('submission_bm25.csv', 'w') as output:
    output.write('QueryId,DocumentId\n')
    for index, row in tqdm(relevance_data.iterrows()):
        query = row['Query']
        id = row['QueryId']
        relevant_documents = row['DocumentId'].split(',')
        data = database[database['id'].isin(list(relevant_documents))].copy()
        data.loc[:, 'content'] = data[['subject', 'content']].astype(str).apply(' '.join, axis=1)
        del data['subject']
        data = OrderedDict(data.set_index('id')['content'].to_dict())

        for key in data:
            data[key] = data[key].split()

        bm25 = BM25Okapi(data.values())
        rankings =  sorted(list(zip(data.keys(), bm25.get_scores(query))), key=lambda x: x[1], reverse=True)[:10]
        for rank in rankings:
            output.write('{},{}\n'.format(id, rank[0]))

        data = None
        

Not a good score 0.35279 ! Maybe because of preprocessing